In [1]:
pip install elasticsearch

   ---------------------------------------- 0.0/524.6 kB ? eta -:--:--
   ---------------------------------------- 524.6/524.6 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [3]:
es = Elasticsearch("http://localhost:9200")

In [10]:
response = es.search(index="sqlserver-cursos", query={"match_all": {}})
print(f"Got {response['hits']['total']['value']} hits:")
for hit in response['hits']['hits']:
    print(f"{hit['_source']['chave']} {hit['_source']['nome']}: {hit['_source']['ende']}")

Got 10000 hits:
2555.0 itajiba: RUA A
2556.0 itajiba: RUA A
2557.0 itajiba: RUA A
2558.0 itajiba: RUA A
2559.0 itajiba: RUA A
2560.0 itajiba: RUA A
2561.0 itajiba: RUA A
2562.0 itajiba: RUA A
2563.0 itajiba: RUA A
2564.0 itajiba: RUA A


In [7]:
print(es.info())

{'name': 'VMDEX80', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'rVyO9Oa_TyK_5SVVg77XwQ', 'version': {'number': '8.13.4', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': 'da95df118650b55a500dcc181889ac35c6d8da7c', 'build_date': '2024-05-06T22:04:45.107454559Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [8]:
# Define the mapping
mappings = {
    "properties": {
        "title_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine",
        }
    }
}

# Create the index
es.indices.create(index="book_index", mappings=mappings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [book_index/ox7lIeSSSAKew0v-cjea0w] already exists')

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

c:\Users\Administrator\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Administrator\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
pip install sentence-transformers==2.7.0

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install -U sentence-transformers
    

Note: you may need to restart the kernel to use updated packages.


In [9]:
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json"
response = urlopen(url)
books = json.loads(response.read())

operations = []
for book in books:
    operations.append({"index": {"_index": "book_index"}})
    # Transforming the title into an embedding using the model
    book["title_vector"] = model.encode(book["title"]).tolist()
    operations.append(book)
es.bulk(index="book_index", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 1198, 'items': [{'index': {'_index': 'book_index', '_id': 'e4Kt3JEB7atN7RTlnPm0', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'fIKt3JEB7atN7RTlnPm0', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'fYKt3JEB7atN7RTlnPm0', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'foKt3JEB7atN7RTlnPm0', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'

In [10]:
def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            publication_date = hit["_source"]["publish_date"]
            score = hit["_score"]
            title = hit["_source"]["title"]
            summary = hit["_source"]["summary"]
            publisher = hit["_source"]["publisher"]
            num_reviews = hit["_source"]["num_reviews"]
            authors = hit["_source"]["authors"]
            pretty_output = f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nPublisher: {publisher}\nReviews: {num_reviews}\nAuthors: {authors}\nScore: {score}"
            print(pretty_output)

In [12]:
response = es.search(
    index="book_index",
    knn={
        "field": "title_vector",
        "query_vector": model.encode("javascript books"),
        "k": 10,
        "num_candidates": 100,
    },
)

pretty_response(response)


ID: g4Kt3JEB7atN7RTlnPm0
Publication date: 2008-05-15
Title: JavaScript: The Good Parts
Summary: A deep dive into the parts of JavaScript that are essential to writing maintainable code
Publisher: oreilly
Reviews: 51
Authors: ['douglas crockford']
Score: 0.8042828

ID: f4Kt3JEB7atN7RTlnPm0
Publication date: 2015-03-27
Title: You Don't Know JS: Up & Going
Summary: Introduction to JavaScript and programming as a whole
Publisher: oreilly
Reviews: 36
Authors: ['kyle simpson']
Score: 0.6989136

ID: gIKt3JEB7atN7RTlnPm0
Publication date: 2018-12-04
Title: Eloquent JavaScript
Summary: A modern introduction to programming
Publisher: no starch press
Reviews: 38
Authors: ['marijn haverbeke']
Score: 0.67969894

ID: e4Kt3JEB7atN7RTlnPm0
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: 0.6206549

I

In [14]:
response = es.search(
    index="book_index",
    knn={
        "field": "title_vector",
        "query_vector": model.encode("javascript books"),
        "k": 10,
        "num_candidates": 100,
        "filter": {"term": {"publisher.keyword": "addison-wesley"}},
    },
)

pretty_response(response)


ID: e4Kt3JEB7atN7RTlnPm0
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: 0.6206549

ID: gYKt3JEB7atN7RTlnPm0
Publication date: 1994-10-31
Title: Design Patterns: Elements of Reusable Object-Oriented Software
Summary: Guide to design patterns that can be used in any object-oriented language
Publisher: addison-wesley
Reviews: 45
Authors: ['erich gamma', 'richard helm', 'ralph johnson', 'john vlissides']
Score: 0.56499225
